In [58]:
import glob
import re
import parselmouth
from parselmouth.praat import call
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [65]:
sung = []
voice = []

f = open(r"C:/Users/15712/Documents/Boston College/Year 4/nlp/final project/features.csv", "w")

f.write("filename,singvspeak,pitch,intensity,f1,f2,vowel_duration,jitter,shimmer,hnr,f1_uw,f1_ah,f1_eh,f1_er,f1_iy" + "\n")


# You will obviously need to change this path below to make it work.
for wav_file in glob.glob(r"C:/Users/15712/Documents/Boston College/Year 4/nlp/final project/data/*.wav"):
    
    #Get jitter,shimmer,hnr
    accent = 0
    filename = re.sub("^.*\/", "", wav_file)
    tg_file = re.sub("wav", "TextGrid", filename)
    textgrid = call("Read from file", tg_file)
    intv = call(textgrid, "Get number of intervals", 2)
    sound = parselmouth.Sound(filename)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", 75, 600)
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
    jitter = 0.0
    shimmer = 0.0
    hnr = 0.0
    count = 0
    print(filename)
    for i in range(1, intv):
        phone = call(textgrid, "Get label of interval", 2, i)
        if re.match('[AEIOU]', phone.upper()):
            vowel_onset = call(textgrid, "Get starting point", 2, i)
            vowel_offset = call(textgrid, "Get end point", 2, i)
            j = call(pointProcess, "Get jitter (local)", vowel_onset, vowel_offset, 0.0001, 0.03, 1.3)
            s = call([sound, pointProcess], "Get shimmer (local)", vowel_onset, vowel_offset, 0.0001, 0.02, 1.3, 1.6)
            h = call(harmonicity, "Get mean", vowel_onset, vowel_offset)
            print(j,s,h)
            if j >= 0 and s >= 0 and h >= 0:
                count += 1
                jitter += j
                shimmer += s
                hnr += h
                
    # get mean F0 (vowels only)
    formant = call(sound, "To Formant (burg)", 0, 5, 5500, 0.025, 50)
    tg_file = re.sub("wav", "TextGrid", wav_file)
    textgrid = call("Read from file", tg_file)
    intv = call(textgrid, "Get number of intervals", 2)
    vowels = 0
    f_zero = 0
    for i in range(1, intv):
        phone = call(textgrid, "Get label of interval", 2, i)
        phone = str(phone)
        if re.match('[AEIOU]', phone.upper()):
            vowels += 1
            vowel_onset = call(textgrid, "Get starting point", 2, i)
            vowel_offset = call(textgrid, "Get end point", 2, i)
            midpoint = (vowel_offset - vowel_onset) / 2
            f_zero += call(formant, "Get value at time", 2, midpoint, "Hertz", "Linear")

    # Get duration, mean pitch, mean intensity
    sound = parselmouth.Sound(wav_file)
    pitch = call(sound, "To Pitch", 0, 75, 600) 
    meanpitch = call(pitch, "Get mean", 0, 0, "Hertz")
    intensity = call(sound, "To Intensity", 75, 0, "yes")
    meanintensity = call(intensity, "Get mean", 0, 0, "energy")
    duration = call(sound, "Get total duration")
    
    # R feature
    temp_name = re.sub(".wav", "", wav_file)
    
    # get mean F1 (vowels only)
    formant = call(sound, "To Formant (burg)", 0, 5, 5500, 0.025, 50)
    tg_file = re.sub("wav", "TextGrid", wav_file)
    textgrid = call("Read from file", tg_file)
    intv = call(textgrid, "Get number of intervals", 2)
    vowels1 = 0
    vowels2 = 0
    vowels3 = 0
    vowels4 = 0
    vowels5 = 0
    vowels6 = 0
    vowels7 = 0
    vowels8 = 0
    vowels9 = 0
    vowels10 = 0
    consonant1 = 0
    f_f1 = 0
    f_f2 = 0
    f_one = 0
    f_three = 0
    f_one_a = 0
    f_one_o = 0
    f_one_e = 0
    f_two_o = 0
    f_one_one = 0
    vowel_duration = 0
    vowel_pitch = 0
    #print(filename)
    for i in range(1, intv):
        phone = call(textgrid, "Get label of interval", 2, i)
        #print(phone)
        if re.match('[AEIOU]', phone.upper()):
            vowels1 += 1
            vowel_onset = call(textgrid, "Get starting point", 2, i)
            vowel_offset = call(textgrid, "Get end point", 2, i)
            midpoint = vowel_onset + ((vowel_offset - vowel_onset) / 2)
            f_f1 += call(formant, "Get value at time", 1, midpoint, "Hertz", "Linear")

            vowels2 += 1
            f_f2 += call(formant, "Get value at time", 2, midpoint, "Hertz", "Linear")

            vowels3 += 1
            f_three += call(formant, "Get value at time", 3, midpoint, "Hertz", "Linear")

            vowels4 += 1
            vowel_duration += vowel_offset - vowel_onset
            
            if 'UW' == phone.upper():
                vowels5+=1
                f_one_a += call(formant, "Get value at time", 1, midpoint, "Hertz", "Linear")
            if 'AH' == phone.upper():
                vowels6+=1
                f_one_o += call(formant, "Get value at time", 1, midpoint, "Hertz", "Linear")
            if 'EH'== phone.upper():
                vowels7+=1
                f_one_e += call(formant, "Get value at time", 1, midpoint, "Hertz", "Linear")
            if 'ER'== phone.upper():
                f_two+=1
                f_three += call(formant, "Get value at time", 1, midpoint, "Hertz", "Linear")
            if 'IY'== phone.upper():
                f_one+=1
                f_one_one += call(formant, "Get value at time", 1, midpoint, "Hertz", "Linear")
                
        avg_f1_uw = 0
        if not(vowels5 == 0):
            avg_f1_uw = f_one_a / vowels5
        avg_f1_ah = 0
        if not(vowels6 == 0):
            avg_f1_ah = f_one_o / vowels6
        avg_f1_eh = 0
        if not(vowels7 == 0):
            avg_f1_eh = f_one_e / vowels7
        avg_f1_er = 0
        if not(f_two == 0):
            avg_f1_er = f_three / f_two
        avg_f1_iy = 0
        if not(f_one == 0):
            avg_f1_iy = f_one_one/ f_one

    # Append all the features to the northeast data list.
    #print(filename)
    #print(filename, vowels, vowels1, vowels2, vowels4, count)
    if 'sung' in filename or 'sing' in filename: 
        #sung.append([meanpitch, meanintensity, avg_f1_a, avg_f1_o, avg_f1_e, vowel_duration/vowels4, f_one/vowels1, f_two/vowels2])
        #f.write(filename[5:] + "," + str(meanpitch) + "," + str(meanintensity) + "," + str(f_zero/vowels) + "," + str(avg_f1_a) + "," + str(avg_f1_o) + "," + str(avg_f1_e) + "," + str(vowel_duration/vowels4) + "," + str(f_one/vowels1) + "," + str(f_two/vowels2) + "," + str(jitter/count) + "," + str(shimmer/count) + "," + str(hnr/count) + ",0\n")
        f.write(filename[5:] + ",0," + str(meanpitch) + "," + str(meanintensity) + "," + str(f_f1/vowels1) + "," + str(f_f2/vowels2) + "," + str(vowel_duration/vowels4) + "," + str(jitter/count) + "," + str(shimmer/count) + "," + str(hnr/count) + "," + str(avg_f1_uw) + "," + str(avg_f1_ah) + "," + str(avg_f1_eh) + "," + str(avg_f1_er) + "," + str(avg_f1_iy) + "\n")
    else:
        #voice.append([meanpitch, meanintensity, avg_f1_a, avg_f1_o, avg_f1_e, vowel_duration/vowels4, f_one/vowels1, f_two/vowels2])
        #f.write(filename[5:] + "," + str(meanpitch) + "," + str(meanintensity) + "," + str(f_zero/vowels) + "," + str(avg_f1_a) + "," + str(avg_f1_o) + "," + str(avg_f1_e) + "," + str(vowel_duration/vowels4) + "," + str(f_one/vowels1) + "," + str(f_two/vowels2) + "," + str(jitter/count) + "," + str(shimmer/count) + "," + str(hnr/count) + ",1\n")
        f.write(filename[5:] + ",1," + str(meanpitch) + "," + str(meanintensity) + "," + str(f_f1/vowels1) + "," + str(f_f2/vowels2) + "," + str(vowel_duration/vowels4) + "," + str(jitter/count) + "," + str(shimmer/count) + "," + str(hnr/count) + "," + str(avg_f1_uw) + "," + str(avg_f1_ah) + "," + str(avg_f1_eh) + "," + str(avg_f1_er) + "," + str(avg_f1_iy) + "\n")
f.close()
    

data\adele-sing-a.wav
0.005035300286058342 0.08694599554883159 15.68575881911836
data\adele-sing-and.wav
0.017096216657703844 0.08867912715475573 9.705446154015945
data\adele-sing-but.wav
0.005598013421788248 0.05779743179095612 15.194935824644691
data\adele-sing-crawling.wav
0.0030898472398401 0.04051304982855753 16.63882157379402
0.028840141563121393 0.18873877539627706 5.394333214667069
data\adele-sing-dry.wav
0.0032340921276721095 0.0482683807150393 14.278322063160466
data\adele-sing-face.wav
0.014056004337874661 0.13851309721213476 13.506500474012036
data\adele-sing-feel.wav
0.0028555209637897874 0.024886265521064193 19.11098642459164
data\adele-sing-i.wav
0.006389434998591309 0.04212065419271172 18.265188808790263
data\adele-sing-in.wav
0.00602324913794134 0.10547166745350638 11.326576489057137
data\adele-sing-love.wav
0.0043506467367395 0.058363090943415366 18.364986536686228
data\adele-sing-make.wav
0.007599523562293063 0.07548413999323722 14.031944173266888
data\adele-sing-my.

data\sing-bad.wav
0.0022554931096466233 0.04893974447766856 15.9474906908496
data\sing-be.wav
0.005012697600242007 0.06085926517632136 8.571289982516936
data\sing-best.wav
0.0028103077318861504 0.03236053658725625 14.732090913463688
data\sing-end.wav
0.0953826573302697 0.1471300223319608 1.4618446069938855
data\sing-feel.wav
0.0008854952912279484 0.04520730505986603 16.28491733332065
data\sing-i.wav
0.006136350972463592 0.04795874708418997 14.230720303648512
data\sing-its.wav
0.03225093335268234 0.09529791656383364 3.3045204873624057
data\sing-news.wav
0.0017484602566584981 0.053909049158744386 15.789996016291598
data\sing-night.wav
0.001856233874942404 0.031882621335845455 18.342020378965213
data\sing-no.wav
0.001713725966706 0.033033701009303175 17.506497773717385
data\sing-one.wav
0.0031595117734822768 0.06307905208584567 14.4430070391011
data\sing-scene.wav
0.002410677862672271 0.030915946060467752 15.549753476440735
data\sing-the.wav
0.004173709648890495 0.015533942355620814 19.56

data\talk-dont.wav
0.006723413394066125 0.05436666982081186 13.488607428901323
data\talk-eyes.wav
0.0992507589052091 0.14396868244530367 3.6977225491693964
data\talk-fail.wav
0.012452931841921798 0.11862255681554341 9.288671701920375
data\talk-has.wav
0.012972838373812374 0.18008410447576287 4.859325701818326
data\talk-hey.wav
0.0071859053115362476 0.06480804216906053 12.772248571442363
data\talk-hope.wav
0.011549899277173198 0.0714656772049177 15.389441828288
data\talk-i.wav
0.024942123408819475 0.16366718604374797 3.6646495147560625
data\talk-lip.wav
0.005159637529559066 0.07202674565215705 17.71777515170315
data\talk-me.wav
0.011985260145662839 0.11774185260474272 12.55209977628123
data\talk-my.wav
0.015058806871923064 0.10801614726297301 10.099320460446844
data\talk-or.wav
0.008115645375103583 0.07109566709703374 15.64576918950902
data\talk-pale.wav
0.017963360743482745 0.14163365875989484 8.003966889732082
data\talk-red.wav
0.006379761701045474 0.12712580292612932 13.7299412592153